In [110]:
import tensorflow;
import os;

os.environ["TL_BACKEND"] = "tensorflow";
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async";

import tensorlayerx;
from tensorflow import convert_to_tensor;
import numpy;

from tensorflow.keras.datasets import mnist;

In [111]:
print(tensorflow.config.list_physical_devices('GPU'));


import tensorflow as tf

# Set memory growth for GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        # Set memory growth
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print("Memory growth set for GPU")
    except RuntimeError as e:
        print(e)

[]


In [112]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data();

# Split the test and val by 50:50
test_val_images_split = numpy.array_split(test_images, 2);
test_val_labels_split = numpy.array_split(test_labels, 2);

test_images = test_val_images_split[0];
test_labels = test_val_labels_split[0];

val_images = test_val_images_split[1];
val_labels = test_val_labels_split[1];

In [113]:
# Recall step 1
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255;
test_images = test_images.reshape((5000, 28, 28, 1)).astype('float32') / 255;
val_images = val_images.reshape((5000, 28, 28, 1)).astype('float32') / 255;


# Recall step 2
train_labels = tensorflow.one_hot(train_labels, depth = 10);
test_labels = tensorflow.one_hot(test_labels, depth = 10);
val_labels = tensorflow.one_hot(val_labels, depth = 10);

In [114]:
from tensorlayerx.nn import Sequential, Conv2d, MaxPool2d, Flatten, Linear, Module, Input, ReLU, Softmax;
from tensorlayerx.model import TrainOneStep;
from tensorlayerx.optimizers import Adam;
from tensorlayerx.losses import cross_entropy_seq;
from tqdm import tqdm;

In [115]:
class CNNModel (Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        self.conv1 = Conv2d(out_channels = 32, kernel_size = (3, 3), stride = (1, 1), in_channels = 1, act = ReLU, name = "conv1");
        # self.pool1 = MaxPool2d(kernel_size = (2, 2), name = "pool1");

        self.conv2 = Conv2d(out_channels = 64, kernel_size = (1, 1), stride = (1, 1), in_channels = 1, act = ReLU, name = "conv2");
        # self.pool2 = MaxPool2d(kernel_size = (2, 2), name = "pool2");

        self.flat = Flatten(name = "flat");

        self.dense = Linear(out_features = 64, in_features = 50176, act = ReLU, name = "dense1");
        self.output = Linear(out_features = 10, in_features = 64, act = Softmax, name = "output")
    
    def forward(self, x):
        x = self.conv1(x);
        # x = self.pool1(x);
    
        x = self.conv2(x);
        # x = self.pool2(x);

        x = self.flat(x);
        x = self.dense(x);

        x = self.output(x);
    
        return x;

    
    def compile(self):
        model = Sequential([
            self.conv1,
            self.pool1,
            self.conv2,
            self.pool2,
            self.flat,
            self.output
        ]);

        print("------------------------------------------------")
        print("               MNIST Tensorlayerx               ");
        print("------------------------------------------------")
        print("Neuron: ", sum(layer.out_features for layer in model if hasattr(layer, 'out_features')))
        print(model);

        return model;

class ForwardPropagation(Module):
    def __init__(self, neural_network):
        super(ForwardPropagation, self).__init__();
        self.net = neural_network;
        self.loss_fn = cross_entropy_seq;

    def forward(self, feature_set, label_set):
        out = self.net(feature_set);
        loss = self.loss_fn(out, label_set);
        return loss;

In [116]:
class ManualTrainer:

    """
    ManualTrainer

    This class is mimicing the Tensorflow's automatic fit. 
    However, since the adversarial training algorithm may requires TrainOneStep.

    @params
        epoch: number of epoch
        batch_size: total data per training minibatch (For every epoch elapsed, train the data with minibatches.)
        model: model to be trained (tensorlayerx.nn.Sequential)
        optimizer: optimizer algorithm to be used for training (tensorlayerx.optimizers)
        loss: loss function to be used for training (tensorlayerx.losses)
        
        ----------------------------------------------------------------
        Every parameters below are defined as array length of 2:
        train_set -> [ [X_set], [Y_set] ]
        val_set -> [ [X_set], [Y_set] ]
    """
    def __init__(self, epoch: int, batch_size: int, model, optimizer, loss, train_set, val_set = []):
        self.epoch = epoch;
        self.batch_size = batch_size;
        self.optimizer = optimizer;
        self.loss = loss;
        self.model = model;
        
        # Dataset
        self.train_set = [
            [train_set[0]],
            [train_set[1]],
        ];

        # If the validation is not set, then the validation set will be using training set instead
        self.val_set = [
            [train_set[0]] if (len(val_set) == 0) else [val_set[0]],
            [train_set[1]] if (len(val_set) == 0) else [val_set[1]],
        ];


        self.average_training_loss = 0.0;
        self.validation_loss = 0.0;
        self.validation_accuracy = 0.0;

        # Train One Step
    
    def fitting(self):
        loss = 0.0;

        feed_forward = TrainOneStep(self.model, optimizer = self.optimizer, train_weights = True);

        # Start of epoch
        for epoch in tqdm(range(self.epoch)):
            epoch_loss = 0.0;

            # Start of batch training
            for batch in range(self.batch_size):
                start_dataset_index = batch * self.batch_size;
                end_dataset_index = (batch + 1) * self.batch_size;

                feature_set = convert_to_tensor(self.train_set[0][start_dataset_index:end_dataset_index]);
                label_set = convert_to_tensor(self.train_set[1][start_dataset_index:end_dataset_index]);

                training_loss_value = feed_forward(feature_set, label_set);

                epoch_loss += training_loss_value;
            # End of batch training
            
            self.average_training_loss = epoch_loss / self.batch_size;

            # The loss now count with validation set 
            loss, accuracy = model.evaluate(self.val_set[0], self.val_set[1]);

            self.validation_loss = loss;
            self.validation_accuracy = accuracy;

            print(f"Epoch {epoch + 1}/{self.epoch} | loss: {self.validation_loss} | accuracy: {self.validation_accuracy}")

        # End of epoch
        return self.model;

In [117]:
cnn = CNNModel();
cnn.init_build(Input(shape = (8, 28, 28, 1)));

forward_propagation = ForwardPropagation(neural_network = cnn);

trainer = ManualTrainer(
    epoch = 5, 
    batch_size = 64, 
    optimizer = Adam(lr = 1e-3),
    loss = cross_entropy_seq,
    model = forward_propagation,
    train_set = [train_images, train_labels],
    val_set = [val_images, val_labels]
);

model = trainer.fitting();
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

[TLX] Conv2d conv1: out_channels : 32 kernel_size: (3, 3) stride: (1, 1) pad: SAME act: ReLU
[TLX] Conv2d conv2: out_channels : 64 kernel_size: (1, 1) stride: (1, 1) pad: SAME act: ReLU
[TLX] Flatten flat:
[TLX] Linear  dense1: 64 ReLU
[TLX] Linear  output: 10 Softmax
[TLX] Input  _inputlayer_17: (8, 28, 28, 1)


  0%|          | 0/5 [00:00<?, ?it/s]